In [5]:
from numpy.linalg import norm
from pandas import read_csv
import pickle
from pymorphy2 import MorphAnalyzer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

morph = MorphAnalyzer()
stops = open('russian.txt').read().split()

In [6]:
data = pd.read_csv('data.csv', sep=';')
vectorizer = TfidfVectorizer(max_features=1000)
tfidf = vectorizer.fit(data['plot'])
X = vectorizer.transform(data['plot']).toarray()
pickle.dump(tfidf, open('tfidf.pickle', 'wb'))
pickle.dump(X, open('tfidf_matrix.pickle', 'wb'))    # https://yadi.sk/d/tbhfd_gJY41ofg

def predict(query):
    query = ' '.join(list(morph.parse(word)[0].normal_form for word in re.findall("[\w]+", query)))
    query = ' '.join([word for word in query.split() if word not in stops])
    vec = vectorizer.transform([query]).toarray().reshape(1000, 1)
    data = read_csv('data.csv', sep=';')
    simil = []
    for vector, film in zip(X, data['film']):
        simil.append((vector@vec/norm(vector)*norm(vec), film))
    simil.sort(reverse=True)
    return simil[:5]

query = ('Ким Ки Тхэк раньше работал водителем, но давно уже потерял работу'
         'и теперь влачит жалкое существование вместе со своей женой Чхун Сук'
         'и двумя детьми выпускного возраста в полуподвальной квартире. Семья'
         'пытается свести концы с концами, складывая из картонок коробки для пиццы.')
#query = 'Сериал про Шерлока Холмса'
predict(query)

[(array([0.34949623]), 'Драйв (фильм, 2011)'),
 (array([0.3224357]), 'Водитель (фильм)'),
 (array([0.30593387]), 'Скрытый цвет вещей'),
 (array([0.30349621]), 'Налево от лифта'),
 (array([0.2923081]), 'Отрыв (фильм, 2007)')]